<a href="https://colab.research.google.com/github/naveen3699-cyber/FMML-Lab/blob/main/backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def fetch_stock_data(stock_symbol, years=1):
    """Fetch historical stock data for a specific company from Yahoo Finance with error handling."""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years * 365)

    try:
        print(f"Fetching data for {stock_symbol}...")
        stock_data = yf.download(stock_symbol, start=start_date, end=end_date, progress=False)
        if stock_data.empty:
            raise ValueError("No data found. Please check the stock symbol.")
        return stock_data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def preprocess_data(stock_data):
    """Prepare data for machine learning model."""
    if stock_data is None:
        return None

    stock_data['Date'] = stock_data.index
    stock_data['Date'] = stock_data['Date'].map(datetime.toordinal)

    return stock_data[['Date', 'Close']].dropna()

def train_and_predict(stock_data):
    """Train a model and predict tomorrow's stock price."""
    if stock_data is None:
        print("No data available for training.")
        return None, None

    X = stock_data[['Date']]
    y = stock_data['Close']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    tomorrow_date = (datetime.now() + timedelta(days=1)).toordinal()
    tomorrow_prediction = model.predict(np.array([[tomorrow_date]]))[0]

    return tomorrow_date, tomorrow_prediction

def display_prediction(tomorrow_date, tomorrow_prediction):
    """Display tomorrow's stock price prediction in text format."""
    if tomorrow_date is None or tomorrow_prediction is None:
        print("No prediction available.")
        return

    readable_date = datetime.fromordinal(tomorrow_date).strftime('%Y-%m-%d')
    print(f"\nPredicted Stock Price for {readable_date}: ${tomorrow_prediction:.2f}")

# User input
stock_symbol = input("Enter a stock symbol (e.g., AAPL): ")
stock_data = fetch_stock_data(stock_symbol, years=5)
processed_data = preprocess_data(stock_data)

# Train and predict
tomorrow_date, tomorrow_prediction = train_and_predict(processed_data)

# Display result in text
display_prediction(tomorrow_date, tomorrow_prediction)

Enter a stock symbol (e.g., AAPL): JPM
Fetching data for JPM...


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Predicted Stock Price for 2025-03-21: $239.17
